In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 라이브러리를 임포트
import os
from xml.etree import ElementTree
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import os
from xml.etree import ElementTree
import json

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []  # 'bsisFacts' 값을 저장할 리스트

    # 지정된 폴더 내의 모든 파일 목록을 얻기 위해 os.listdir()를 사용
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)  # 파일 경로 생성
        if os.path.isfile(file_path) and filename.endswith('.json'):
            # 파일인지 확인하고 확장자가 '.json'인 파일만 처리
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    # 'bsisFacts'가 'facts' 안에 존재하는 경우
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)  # 리스트 확장

    return bsisFacts_list


# 폴더 경로 지정 (예시 경로입니다, 실제 경로로 변경해야 합니다)
civil_case_directory = '/content/drive/MyDrive/skt/project/law/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/skt/project/law/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 단어 변환
def replace_words_in_texts(texts, word_replacements):
    replaced_texts = []
    for text in texts:
        for old_word, new_word in word_replacements.items():
            text = text.replace(old_word, new_word)
        replaced_texts.append(text)
    return replaced_texts

# 단어 변환 매핑
word_replacements = {
    "형법": "법",
    "민법": "법",
    "형사": "소송",
    "민사": "소송",
    "피고": "사람",
    "피고인": "사람"
}

# 데이터 변환
civil_texts_r = replace_words_in_texts(civil_texts, word_replacements)
criminal_texts_r = replace_words_in_texts(criminal_texts, word_replacements)


# 민사 및 형사 텍스트 데이터를 하나의 데이터셋으로 결합
texts_all = civil_texts_r + criminal_texts_r  # 전체 텍스트 데이터
labels = [0] * len(civil_texts_r) + [1] * len(criminal_texts_r)  # 민사: 0, 형사: 1

In [ ]:
# bert model 설계

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 전처리된 텍스트 데이터와 레이블을 결합합니다.
texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = [0] * len(civil_texts) + [1] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저를 로드합니다.
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# BERT에 맞게 데이터를 인코딩합니다.
encoded_data = tokenizer.batch_encode_plus(
    texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = torch.tensor(labels)

# 데이터셋을 생성합니다.
dataset = TensorDataset(input_ids, attention_masks, labels)

# 데이터를 훈련 세트와 검증 세트로 분리합니다.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# 데이터 로더를 생성합니다.
batch_size = 4

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),
    batch_size = batch_size
)

# BERT 분류 모델을 로드합니다.
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 옵티마이저를 설정합니다.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# 훈련 루프를 정의합니다.
epochs = 3
# 모델을 device로 보냅니다.
model.to(device)
for epoch in range(0, epochs):
    # 훈련 단계
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 평가 단계
    model.eval()
    for batch in validation_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs[0]
            logits = outputs[1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score

# 훈련 루프를 정의합니다.
epochs = 3
for epoch in range(epochs):
    # 훈련 단계
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 평가 단계
    model.eval()
    val_loss = 0
    predictions, true_labels = [], []
    for batch in validation_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs.loss
        logits = outputs.logits
        val_loss += loss.item()

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].to('cpu').numpy()

        # 예측 값 저장
        predictions.append(logits)
        true_labels.append(label_ids)

    # 평가 결과 계산
    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(true_labels, predicted_labels)

    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Training loss: {total_loss / len(train_dataloader)}')
    print(f'Validation loss: {val_loss / len(validation_dataloader)}')
    print(f'Accuracy: {accuracy}')


Epoch 1/3
Training loss: 0.014665082336813655
Validation loss: 0.060964139303125976
Accuracy: 0.9821428571428571
Epoch 2/3
Training loss: 0.018569857100720538
Validation loss: 0.1117713322122914
Accuracy: 0.9761904761904762
Epoch 3/3
Training loss: 0.001327549474288909
Validation loss: 0.12505947779988805
Accuracy: 0.9761904761904762


In [ ]:
# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/skt/project/law/test/2017가합202415.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)

    if 'facts' in  new_example_text and 'bsisFacts' in  new_example_text['facts']:
        # 'bsisFacts'가 'facts' 안에 존재하는 경우
        bsisFacts =  new_example_text['facts']['bsisFacts']

pred_texts_r = replace_words_in_texts(bsisFacts, word_replacements)
print(pred_texts_r)

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts_r,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])


["사람는 000에 있는 000병원(이하 '이 사건 병원'이라 한다)을 운영하는 의료법인이고, 의사 소외 00은 이사건 병원에서 근무하던 신경외과 전문의이며, 망 소외 00(이하 '망인'이라 한다)는 2016. 11. 16. 이 사건 병원에서 의사 소외 00로부터 진료를 받은 사람이고, 원고는 망인의 아들이자 유일한 상속인이다.", '망인은 2009년 무렵 0000병원에서 심혈관계 질환의 일종인 심근경색 진단을 받아 스텐트(혈관을 확장하는 구조물) 시술을 받은 후 심근경색 치료제를 장기간 복용하고 있었다.', '망인은 2016. 11. 15.경 오른쪽 발목을 다쳐 2016. 11. 16. 13:30 무렵 이 사건 병원에 내원하였다.', '위 병원의 신경외과 전문의 소외 00은 망인을 진찰하고 엑스레이 촬영 등 검사를 한 다음 오른쪽 발목 부위 인대손상으로 진단하고, 같은 날 14:26 주사약으로 비스테로이드성 소염진통제(NSAID)의 일종인 디클로페낙 성분의 주사제 로페낙-주 2ml(디클로페낙 나트륨)를, 먹는 약으로 엔클로페낙정(아세클로페낙), 에페신정(에페리손염산염), 케이비피드정(레바미피드)을 처방하였다.', '위와 같은 의사 소외 00의 처방에 따라 이 사건 병원의 간호사인 소외 00은 망인에게 유니페낙 2cc를 근육주사하였는데, 의사소외 00이 망인에게 처방한 로페낙과 간호사 소외 00이 실제로 망인에게 주사한 유니페낙은 모두 디클로페낙 나트륨을 주성분으로 하는 비스테로이드성 소염진통 주사제로 성분은 동일하고 제품명만 다르다.', '망인은 이 사건 병원에서 위와 같이 주사를 맞은 후 처방약을 조제하려고 병원 가까이에 있는 0에 가서 약사에게 이사건 쪽지를 보여주면서 처방전에 있는 약과 위 쪽지에 적힌 약의 성분이 같은지 를 묻고, 약사가 성분이 거의 비슷하다고 하자, 나는 이 약을 먹으면 큰일 난다 고 말하면서 동행하였던 망인의 동거인 소외 00와 함께 처방전 변경을 위하여 위 병원으로 되돌아갔다.', '망인은 오른쪽 발목을 다친 상태여서 걸음이 느렸

In [ ]:
# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/skt/project/law/test/2019고합426.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)

    if 'facts' in  new_example_text and 'bsisFacts' in  new_example_text['facts']:
        # 'bsisFacts'가 'facts' 안에 존재하는 경우
        bsisFacts =  new_example_text['facts']['bsisFacts']

pred_texts_r = replace_words_in_texts(bsisFacts, word_replacements)
print(pred_texts_r)

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts_r,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])

['사람인은 조현병으로 인하여 사물을 변별할 능력이나 의사를 결정할 능력이 미약한 상태에서 다음과 같은 범행들을 저질렀다.', '사람인은 2018. 11. 21. 23:25경 0000에 있는 피해자 00이 근무하는 0 식당에서, 마치 대금을 지불할 것처럼 행세하면서 피해자에게 술과 음식 등을 주문하였다.', '그러나 사실 사람인은 수중에 현금이 없었으므로 피해자로부터 술과 음식을 제공받더라도 그 대금을 지불할 의사나 능력이 없었다.', '그럼에도 사람인은 위와 같이 피해자를 기망하여 이에 속은 피해자로부터 즉석에서 시가 8,000원 상당의 곰탕 한 그릇, 시가 4,000원 상당의 소주 1병 등 합계 12,000원 상당의 술과 음식을 제공 받았다.', '이로써 사람인은 피해자를 기망하여 재물을 교부 받았다.', '사람인은 전항 기재 일시 장소에서, 피해자가 사람인에게 술과 음식 값의 지불을 요구하자 이에 화가 나, 주먹으로 위 식당 카운터에 설치되어 있던 포스(POS)단말기 모니터를 밀어 카운터 밑으로 떨어지게 하고, 양손 주먹으로 피해자의 얼굴 부위를 2회 때렸다.', '이로써 사람인은 피해자를 폭행함과 동시에 위력으로써 피해자의 식당 영업업무를 방해하였다.', '사람인은 조현병으로 인하여 사물을 변별할 능력이나 의사를 결정할 능력이 미약한 상태에서 위와 같은 금고 이상의 형에 해당하는 죄를 지었고, 치료감호시설에서 치료를 받을 필요가 있으며, 재범의 위험성도 있다.']
예측된 분류: 형사


In [ ]:
# 직접 입력 코드

# bsisFacts = input("법률 상황을 말해주세요!")
bsisFacts = ['내가 차를 운전하다가 졸아서 사람을 쳐버렸어, 그 사람은 죽지는 않았고 중환자실에 입원했어'] # 결과 형사
# bsisFacts = ['내가 차를 운전하다가 졸아서 사람을 쳐버렸어.', '그 사람은 죽지는 않았고 중환자실에 입원했어'] # 결과 형사
# bsisFacts = ['윗집 아져씨가 시끄럽게 해서 화가 났어', '망치를 들고가서 위협만 하려 했는데 한대 때려버렸어','아져씨가 죽었어'] # 결과 민사

pred_texts_r = replace_words_in_texts(bsisFacts, word_replacements)

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts_r,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])

예측된 분류: 형사
